# Hikaru Game Analysis

In [ ]:
import chess.pgn as ch
import pandas as pd
import re

# --- Path to your PGN file ---
pgn_path = "GMHikaruOnTwitch.pgn"

# --- Open and read the first game in the PGN file ---
with open(pgn_path, encoding="utf-8") as pgn:
    game = ch.read_game(pgn)

# --- Metadata DataFrame ---
metadata = pd.DataFrame([dict(game.headers)])
metadata["Game_ID"] = 1
metadata.set_index("Game_ID", inplace=True)

# --- Extract Moves and Clock Times ---
move_data = []
node = game

move_num = 1
white_move = None
white_time = None
black_move = None
black_time = None

clk_pattern = re.compile(r"\[%clk ([0-9:.\-]+)\]")

while node.variations:
    next_node = node.variation(0)
    move_san = next_node.san()
    comment = next_node.comment

    clk_match = clk_pattern.search(comment)
    clk_time = clk_match.group(1) if clk_match else None

    # White to move = True, so store white move first
    if node.board().turn:
        white_move = move_san
        white_time = clk_time
    else:
        black_move = move_san
        black_time = clk_time
        move_data.append({
            "move_number": move_num,
            "white_move": white_move,
            "white_time": white_time,
            "black_move": black_move,
            "black_time": black_time
        })
        move_num += 1

    node = next_node

# --- Moves DataFrame ---
move_df = pd.DataFrame(move_data)
move_df.set_index("move_number", inplace=True)

# --- Display ---
print("=== METADATA ===")
print(metadata.T)
print("\n=== MOVES (first 10) ===")
print(move_df.head(10))

=== METADATA ===
Game_ID                                                          1
Event                                                   Live Chess
Site                                                     Chess.com
Date                                                    2020.07.17
Round                                                            -
White                                             GMHikaruOnTwitch
Black                                               subhangagrawal
Result                                                         1-0
CurrentPosition         8/4k2p/p1p5/1pP2B2/1P1K1P2/6P1/P6P/8 b - -
Timezone                                                       UTC
ECO                                                            A00
ECOUrl           https://www.chess.com/openings/Vant-Kruijs-Ope...
UTCDate                                                 2020.07.17
UTCTime                                                   15:19:15
WhiteElo                                     